<a href="https://colab.research.google.com/github/mzohaibnasir/NeuralNotes/blob/main/04_deepDiveIntoBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#| default_exp datasets


In [286]:
#|export

from __future__ import annotations
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc

from torch.utils.data import default_collate

# from miniai.training import *

In [287]:
# ! pip install datasets

In [288]:
import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path

from torch import tensor,nn,optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
from fastcore.test import test_close

In [289]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [290]:
import logging
logging.disable(logging.WARNING)


# Hugging Face Datasets

In [291]:
"""Use the load_dataset_builder() function to load a dataset builder and inspect a dataset’s attributes without committing to downloading it:"""
name = "fashion_mnist"
from datasets import load_dataset_builder
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)




In [292]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [293]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31304107, num_examples=60000, shard_lengths=None, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5235060, num_examples=10000, shard_lengths=None, dataset_name='fashion_mnist')}

In [294]:
dsd = load_dataset(name)
dsd # DatasetDict  -  dataset dictionary

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [295]:
train, test = dsd['train'], dsd['test']   # fetching element from DatasetDict using keys

train.shape, test.shape

((60000, 2), (10000, 2))

In [296]:
train[0], test[0]  # here we are getting dictionaries, not tuple

({'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  'label': 9},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  'label': 9})

In [297]:
x , y = 'image' ,'label' # for
x , y

('image', 'label')

In [298]:
x, y = ds_builder.info.features
x,y

('image', 'label')

In [299]:
img = train[0][x]
img

In [300]:
train[0][y]

9

In [301]:
train[:5][x], train[:5][y]

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0])

In [302]:
featy = train.features[y]  # label names
featy

ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [303]:
xb,yb = train[:5][x] , train[:5][y]
xb,yb

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0])

In [304]:
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [305]:
train['label'][:5]

[9, 0, 0, 3, 0]

In [306]:
# collate function will return dictionary


def collate_fn(b):
  return {
      x:torch.stack([
          TF.to_tensor(o[x]) for o in b
      ]),

      y:tensor([o[y] for o in b])
  }

In [307]:
dl = DataLoader(train, collate_fn=collate_fn, batch_size=16)
b = next(iter(dl))

b[x].shape, b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [308]:
def transforms(b):  # we can fo collate_fn using transform
  b[x] = [TF.to_tensor(o) for o in b[x]]  # no satcking # pytorch already knows how to collate dictionaries of tenaors
  return b

In [309]:
tds = train.with_transform(transforms) # with_transform expects transforms to return value i.e. not an inplace function  # it `transforms` to every element
# whenever __getitem__ will be called it will call`transforms` on fly  # means its not cached
dl = DataLoader(tds, batch_size=16)  # so data loader can be created without collate function because pytorch collates dicts but default
b = next(iter(dl))
b[x].shape, b[y]  # x - img, y - label


(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [310]:
def _transformi(b):
  b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]
"""
this inplace function does not return results but `with_transform` need `transforms` to return data
"""

'\nthis inplace function does not return results but `with_transform` need `transforms` to return data\n'

In [311]:
def inplace(f):
  def _f(b):
    f(b)
    return b
  return _f

In [312]:
transformi = inplace(_transformi)

In [313]:
r = train.with_transform(transformi)[0]
r[x].shape, r[y]

(torch.Size([784]), 9)

In [314]:
@inplace
def transformi(b):
  b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [315]:
tdsf = train.with_transform(transformi)
r = tdsf[0]
r[x].shape, r[y]

(torch.Size([784]), 9)

In [316]:
d= dict(
    a=1,
    b=2,
    c=3
)

In [317]:
ig = itemgetter('a','c')
ig(d)

(1, 3)

In [318]:
class D:
  def __getitem__(self, k):
    return 1 if k=='a' else 2 if k=='b' else 3

In [319]:
d=D()
ig(d)

(1, 3)

In [320]:
list(tdsf.features)

['image', 'label']

In [321]:
batch = dict(a=[1], b=[2]), dict(a=[3], b=[4])
default_collate(batch) # default_collate takes the matching keys and collate the values

{'a': [tensor([1, 3])], 'b': [tensor([2, 4])]}

In [322]:
#|export


# we will be using this function too much

def collate_dict(ds):  # converting dict into tuple

  get = itemgetter(*ds.features)
  def _f(b): return (get(default_collate(b)))
  # breakpoint()

  return _f

In [323]:
dlf = DataLoader(tdsf, batch_size=4, collate_fn=collate_dict(tdsf))
xb,yb = next(iter(dlf))
xb.shape,  yb

(torch.Size([4, 784]), tensor([9, 0, 0, 3]))